<a href="https://colab.research.google.com/github/cyuancheng/Stock_Prediction/blob/master/ETFSignalChart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Interactive Stock Charts with Moving Averages

In [0]:
print('Installing Holoviews for charting....')
!pip install "holoviews[recommended]"

In [0]:

import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.dates as mdates
import seaborn as sns

from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from bokeh.io import output_notebook
from bokeh.transform import factor_cmap, factor_mark
from bokeh.layouts import column
from bokeh.models import HoverTool
from bokeh.models import ColumnDataSource, RangeTool

%matplotlib inline


In [0]:

def create_signal(data, short, long, threshold):
  
  temp = data
  
  temp['date'] = np.array(data.index, dtype=np.datetime64)
  temp['pct_returns'] = temp.adjusted_close.pct_change()

  temp['ema_s'] = data.adjusted_close.ewm(span=short).mean()
  temp['ema_l'] = data.adjusted_close.ewm(span=long).mean()
 
  temp['ema_ratio'] = temp.ema_s/temp.ema_l - 1
  temp['signal'] = np.where(temp.ema_ratio >= threshold, 1, np.where(temp.ema_ratio < 0, -1, 0))
  temp['cross'] = np.where(((temp.ema_ratio >= 0) & (temp.ema_ratio.shift(1) < 0)) | ((temp.ema_ratio < 0) & (temp.ema_ratio.shift(1) >= 0)), 1,0)
  temp['buy_price'] = np.where(temp.signal == 1, temp.adjusted_close,0)
  temp['cross_price'] = temp.cross*temp.ema_s

  return temp


def chart_signal(data,etf):
  
  # render chart
  output_notebook()


  # Create Bokeh Data

  etf_source = data
  etf_source['signal_size'] =  np.where(data.signal == 1,2.5,0)
  etf_source['cross_size'] =  np.where(data.cross == 1,12,0)
  etf_source = ColumnDataSource(etf_source)
  dates = np.array(data['date'], dtype=np.datetime64)
  y_min = data.adjusted_close.min()-25
  y_max =  data.adjusted_close.max()+25

  # tool tips

  hover = HoverTool(
       tooltips=[
          ('date', '@date{%datetime}'            ),
          ('adjusted_close', '$@{adjusted_close}{%0.1f}' ), # use @{ } for field names with spaces
          ],

       formatters={
          'date' : 'datetime', # use 'datetime' formatter for 'date' field
          'adjusted_close' : 'printf',   # use 'printf' formatter for 'adj close' field
                                    # use default 'numeral' formatter for other fields
      },

      # display a tooltip whenever the cursor is vertically in line with a glyph
        mode='vline'
      )


  # Create a Bokeh figure

  p1 = figure(
      tools = ["pan, zoom_out, zoom_in, reset, save, hover"],
      title = etf+" ETF Chart - First 1000 days", 
      x_axis_label='',  
      x_range=(dates[0], dates[1000]), x_axis_type="datetime",
      y_axis_label='', 
      #y_range=[y_min, y_max],
      width=1000, height=450
  )

  # add some renderers
  p1.line('date','adjusted_close', source=etf_source, line_color = 'grey', legend="Adj Close")
  p1.line('date', 'ema_s', source=etf_source, line_color = 'tomato', legend="EMA Short")
  p1.line('date', 'ema_l', source=etf_source, line_color = 'teal', legend="EMA Long")

  p1.circle('date', 'buy_price', source=etf_source, size='signal_size', color = 'purple', alpha=0.75)
  p1.cross('date', 'cross_price', source=etf_source, size='cross_size', color = 'black', fill_color='white', alpha=0.8)

  p1.legend.location = "top_left"

  p2 = figure(
      tools = ["pan, zoom_out, zoom_in, reset, save, hover"],
      title = etf+" ETF Chart - Last 1000 days", 
      x_axis_label='',  
      x_range=(dates[len(dates)-1000], dates[len(dates)-1]), x_axis_type="datetime",
      y_axis_label='', 
      #y_range=[y_min, y_max],
      width=1000, height=450
  )

  # add some renderers
  p2.line('date','adjusted_close', source=etf_source, line_color = 'grey', legend="Adj Close")
  p2.line('date', 'ema_s', source=etf_source, line_color = 'tomato', legend="EMA Short")
  p2.line('date', 'ema_l', source=etf_source, line_color = 'teal', legend="EMA Long")

  p2.circle('date', 'buy_price', source=etf_source, size='signal_size', color = 'purple', alpha=0.75)
  p2.cross('date', 'cross_price', source=etf_source, size='cross_size', color = 'black', fill_color='white', alpha=0.8)

  p2.legend.location = "top_left"

 
  return  show(column(p1,p2))


### QQQ ETF

In [0]:
# Upload QQQ file

from google.colab import files
uploaded = files.upload()

Saving AlphaVantageDaily_adjusted_qqq.csv to AlphaVantageDaily_adjusted_qqq (2).csv


In [0]:
# Read ETF File

qqq = pd.read_csv('AlphaVantageDaily_adjusted_qqq.csv', infer_datetime_format=True,  index_col=['timestamp'])
qqq.sort_index(inplace=True)
qqq.head(10)


,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient
timestamp,,,,,,,,
1999-03-10,102.3,102.3,100.60,102.10,44.9877,2616000,0.0,1.0
1999-03-11,102.9,103.5,100.60,102.60,45.2080,4844300,0.0,1.0
1999-03-12,102.3,102.3,99.31,100.10,44.1065,4371800,0.0,1.0
1999-03-15,100.9,103.1,99.81,103.00,45.3843,3184500,0.0,1.0
1999-03-16,103.4,104.3,102.30,103.90,45.7808,2452900,0.0,1.0
1999-03-17,103.9,104.0,102.80,103.10,45.4283,1982500,0.0,1.0
1999-03-18,103.0,105.2,103.00,105.10,46.3096,2424200,0.0,1.0
1999-03-19,106.5,106.5,102.40,102.40,45.1199,3580200,0.0,1.0
1999-03-22,102.9,103.1,101.00,101.20,44.5912,2512400,0.0,1.0


In [0]:
qqq.tail(8)

,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient
timestamp,,,,,,,,
2019-06-18,185.88,187.72,185.4291,186.41,185.9992,48177301,0.0000,1.0
2019-06-19,186.73,187.53,185.5700,187.11,186.6976,31672855,0.0000,1.0
2019-06-20,189.72,189.76,187.1585,188.85,188.4338,40921964,0.0000,1.0
2019-06-21,188.57,189.77,188.1736,188.57,188.1544,39980418,0.0000,1.0
2019-06-24,188.52,188.72,188.0600,188.16,188.1600,19375205,0.4156,1.0
2019-06-25,188.06,188.14,184.6500,184.93,184.9300,33921112,0.0000,1.0
2019-06-26,186.13,187.33,185.4900,185.79,185.7900,22512628,0.0000,1.0
2019-06-27,186.47,186.94,185.9900,186.72,186.7200,15889915,0.0000,1.0


In [0]:
# Calculate exponential weighted moving averages

qqq_signal = create_signal(qqq, 10, 40, 0.01)
print('Signal:', qqq_signal.groupby('signal').size())
print('Trend Cross:', qqq_signal.groupby(['cross','signal']).size())
qqq_signal[15:55]

Signal: signal
-1    1744
 0     828
 1    2537
dtype: int64
Trend Cross: cross  signal
0      -1        1671
        0         756
        1        2536
1      -1          73
        0          72
        1           1
dtype: int64


,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient,date,pct_returns,ema_s,ema_l,ema_ratio,signal,cross,buy_price,cross_price
timestamp,,,,,,,,,,,,,,,,,
1999-03-31,108.30,108.4,104.60,104.90,46.2215,4128300,0.0,1.0,1999-03-31,-0.015023,45.873938,45.515332,0.007879,0,0,0.0000,0.000000
1999-04-01,108.00,108.1,105.40,107.10,47.1908,4357900,0.0,1.0,1999-04-01,0.020971,46.121537,45.658052,0.010151,1,0,47.1908,0.000000
1999-04-05,108.90,111.2,108.20,111.00,48.9093,2848600,0.0,1.0,1999-04-05,0.036416,46.642466,45.925273,0.015617,1,0,48.9093,0.000000
1999-04-06,110.90,112.4,109.90,111.20,48.9974,3761200,0.0,1.0,1999-04-06,0.001801,47.080306,46.169609,0.019725,1,0,48.9974,0.000000
1999-04-07,112.30,112.5,107.70,110.10,48.5127,5961800,0.0,1.0,1999-04-07,-0.009892,47.345535,46.350402,0.021470,1,0,48.5127,0.000000
1999-04-08,109.90,111.5,107.60,110.90,48.8652,3976200,0.0,1.0,1999-04-08,0.007266,47.625984,46.539090,0.023354,1,0,48.8652,0.000000
1999-04-09,110.80,111.9,110.40,111.60,49.1736,2473400,0.0,1.0,1999-04-09,0.006311,47.910815,46.731703,0.025232,1,0,49.1736,0.000000
1999-04-12,109.00,111.6,108.90,110.90,48.8652,5467000,0.0,1.0,1999-04-12,-0.006272,48.086074,46.883982,0.025640,1,0,48.8652,0.000000
1999-04-13,111.60,111.8,108.50,109.40,48.2043,4981100,0.0,1.0,1999-04-13,-0.013525,48.107745,46.976137,0.024089,1,0,48.2043,0.000000


In [0]:
qqq_signal.describe().T

,count,mean,std,min,25%,50%,75%,max
open,5109.0,7.438699e+01,4.493983e+01,1.994000e+01,3.939000e+01,5.538000e+01,1.052900e+02,2.298000e+02
high,5109.0,7.501415e+01,4.524498e+01,2.053000e+01,3.968500e+01,5.587000e+01,1.059199e+02,2.329000e+02
low,5109.0,7.365319e+01,4.456079e+01,1.976000e+01,3.900000e+01,5.479000e+01,1.042100e+02,2.270000e+02
close,5109.0,7.437047e+01,4.495419e+01,2.006000e+01,3.935000e+01,5.538000e+01,1.051200e+02,2.300000e+02
adjusted_close,5109.0,6.647846e+01,4.246551e+01,1.767780e+01,3.507370e+01,4.896710e+01,9.014090e+01,1.906888e+02
volume,5109.0,6.996211e+07,5.292414e+07,1.097837e+06,3.017664e+07,6.009376e+07,9.444580e+07,6.165787e+08
dividend_amount,5109.0,2.142983e-03,2.436519e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.206000e-01
split_coefficient,5109.0,1.000196e+00,1.399046e-02,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00
pct_returns,5108.0,4.311937e-04,1.749012e-02,-9.286480e-02,-6.582319e-03,9.695712e-04,7.948560e-03,1.684125e-01
ema_s,5109.0,6.635479e+01,4.229304e+01,1.827115e+01,3.501025e+01,4.904359e+01,9.026108e+01,1.889475e+02


In [0]:
qqq_signal.groupby('signal')['pct_returns','ema_ratio'].describe().T

signal                      -1           0            1
pct_returns count  1744.000000  827.000000  2537.000000
            mean     -0.000556    0.000642     0.001041
            std       0.024062    0.012071     0.013022
            min      -0.089558   -0.046583    -0.092865
            25%      -0.012426   -0.005641    -0.004067
            50%      -0.000079    0.000928     0.001375
            75%       0.011020    0.007824     0.006882
            max       0.168412    0.044075     0.123595
ema_ratio   count  1744.000000  828.000000  2537.000000
            mean     -0.030752    0.005216     0.026783
            std       0.032442    0.002881     0.015123
            min      -0.170239    0.000000     0.010007
            25%      -0.038783    0.002794     0.016486
            50%      -0.019980    0.005410     0.023543
            75%      -0.008094    0.007617     0.031432
            max      -0.000026    0.009994     0.114684

In [0]:
chart_signal(qqq_signal, 'QQQ')

Loading BokehJS ...

### SPY ETF

In [0]:
# Upload QQQ file

from google.colab import files
uploaded = files.upload()

Saving AlphaVanatageDaily_adjusted_spy.csv to AlphaVanatageDaily_adjusted_spy.csv


In [0]:
# Read ETF File

spy = pd.read_csv('AlphaVanatageDaily_adjusted_spy.csv', infer_datetime_format=True,  index_col=['timestamp'])
spy.sort_index(inplace=True)
spy.head(10)

,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient
timestamp,,,,,,,,
1998-01-02,97.3125,97.6562,96.5312,97.5625,66.1219,2360900,0.0,1.0
1998-01-05,97.8437,98.4375,96.7812,97.7812,66.2702,4191800,0.0,1.0
1998-01-06,97.2500,97.2812,96.1875,96.2187,65.2112,3154900,0.0,1.0
1998-01-07,96.0937,96.7187,95.2187,96.4687,65.3806,4424200,0.0,1.0
1998-01-08,96.3125,96.3125,95.3750,95.6250,64.8088,3831000,0.0,1.0
1998-01-09,95.2500,95.5000,91.9062,92.3125,62.5638,10258800,0.0,1.0
1998-01-12,91.1250,94.1875,90.9062,94.0000,63.7075,12097900,0.0,1.0
1998-01-13,94.6250,95.3750,94.2187,95.3125,64.5970,5224900,0.0,1.0
1998-01-14,95.6875,95.9687,94.7187,95.7500,64.8935,3770400,0.0,1.0


In [0]:
# Calculate exponential weighted moving averages

spy_signal = create_signal(spy, 12, 26, 0.01)
print('Signal:', spy_signal.groupby('signal').size())
print('Trend Cross:', spy_signal.groupby(['cross','signal']).size())
spy_signal[100:150]


Signal: signal
-1    1878
 0    2407
 1    1121
dtype: int64
Trend Cross: cross  signal
0      -1        1784
        0        2313
        1        1121
1      -1          94
        0          94
dtype: int64


,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient,date,pct_returns,ema_s,ema_l,ema_ratio,signal,cross,buy_price,cross_price,signal_size,cross_size
timestamp,,,,,,,,,,,,,,,,,,,
1998-05-28,109.8750,110.3750,109.0625,110.1250,74.8489,4907600,0.000,1.0,1998-05-28,0.004561,75.274179,75.311616,-0.000497,-1,1,0.0000,75.274179,0.0,0
1998-05-29,110.6250,110.8125,109.0312,109.0312,74.1055,4772600,0.000,1.0,1998-05-29,-0.009932,75.094382,75.222239,-0.001700,-1,0,0.0000,0.000000,0.0,0
1998-06-01,108.9687,110.2187,108.5625,109.5312,74.4454,6092200,0.000,1.0,1998-06-01,0.004587,74.994539,75.164674,-0.002264,-1,0,0.0000,0.000000,0.0,12
1998-06-02,110.0000,110.3437,109.1562,109.6250,74.5091,6701100,0.000,1.0,1998-06-02,0.000856,74.919856,75.116097,-0.002613,-1,0,0.0000,0.000000,0.0,0
1998-06-03,109.8750,110.1875,107.8750,107.8750,73.3197,6445100,0.000,1.0,1998-06-03,-0.015963,74.673678,74.982989,-0.004125,-1,0,0.0000,0.000000,0.0,0
1998-06-04,108.2500,110.0625,108.0625,109.8750,74.6790,6566300,0.000,1.0,1998-06-04,0.018539,74.674497,74.960465,-0.003815,-1,0,0.0000,0.000000,0.0,0
1998-06-05,110.3750,112.0000,109.8750,112.0000,76.1233,8463200,0.000,1.0,1998-06-05,0.019340,74.897390,75.046624,-0.001989,-1,0,0.0000,0.000000,0.0,0
1998-06-08,111.9375,112.4687,111.6562,111.8750,76.0384,4159900,0.000,1.0,1998-06-08,-0.001115,75.072930,75.120107,-0.000628,-1,0,0.0000,0.000000,0.0,12
1998-06-09,111.7187,112.4218,111.4062,112.2187,76.2720,2725300,0.000,1.0,1998-06-09,0.003072,75.257402,75.205452,0.000691,0,1,0.0000,75.257402,0.0,0


In [0]:
spy_signal.groupby('signal')['pct_returns','ema_ratio'].describe().T

signal                      -1            0            1
pct_returns count  1878.000000  2406.000000  1121.000000
            mean     -0.000208     0.000377     0.001217
            std       0.016794     0.008435     0.008956
            min      -0.096644    -0.057193    -0.041915
            25%      -0.009301    -0.003704    -0.002987
            50%       0.000433     0.000573     0.000873
            75%       0.008451     0.004881     0.005927
            max       0.139608     0.072090     0.039742
ema_ratio   count  1878.000000  2407.000000  1121.000000
            mean     -0.010010     0.005232     0.014131
            std       0.010634     0.002724     0.003644
            min      -0.074721     0.000000     0.010008
            25%      -0.013018     0.003002     0.011412
            50%      -0.007235     0.005414     0.013191
            75%      -0.002841     0.007512     0.015499
            max      -0.000013     0.009989     0.030252

In [0]:
spy_signal.describe().T

,count,mean,std,min,25%,50%,75%,max
open,5406.0,1.507627e+02,5.266238e+01,67.950000,1.140644e+02,1.328596e+02,1.839300e+02,2.960400e+02
high,5406.0,1.516505e+02,5.271521e+01,70.000000,1.148025e+02,1.336800e+02,1.844275e+02,2.963100e+02
low,5406.0,1.497596e+02,5.260083e+01,67.100000,1.132000e+02,1.317628e+02,1.828850e+02,2.937600e+02
close,5406.0,1.507530e+02,5.267196e+01,68.110000,1.140050e+02,1.327750e+02,1.838825e+02,2.958600e+02
adjusted_close,5406.0,1.268769e+02,6.000392e+01,55.006300,8.517663e+01,1.013340e+02,1.650708e+02,2.944263e+02
volume,5406.0,1.034399e+08,9.961119e+07,67900.000000,3.557735e+07,7.537895e+07,1.437105e+08,8.708580e+08
dividend_amount,5406.0,1.105340e-02,9.520128e-02,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,1.435400e+00
split_coefficient,5406.0,1.000000e+00,0.000000e+00,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
pct_returns,5405.0,3.479251e-04,1.210438e-02,-0.096644,-4.845036e-03,6.476799e-04,5.994665e-03,1.396077e-01
ema_s,5406.0,1.266482e+02,5.977076e+01,58.582301,8.520501e+01,1.010910e+02,1.641402e+02,2.906304e+02


In [0]:
chart_signal(spy_signal, 'SPY')

Loading BokehJS ...

### IWM


In [0]:
# Upload QQQ file

from google.colab import files
uploaded = files.upload()

Saving AlphaVantageDaily_adjusted_iwm.csv to AlphaVantageDaily_adjusted_iwm.csv


In [0]:
# Read ETF File

etf = pd.read_csv('AlphaVantageDaily_adjusted_iwm.csv', infer_datetime_format=True,  index_col=['timestamp'])
etf.sort_index(inplace=True)
etf.head(10)

,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient
timestamp,,,,,,,,
2000-05-26,91.06,91.44,90.63,91.44,35.6037,37400,0.0,1.0
2000-05-30,92.75,94.81,92.75,94.81,36.9159,28800,0.0,1.0
2000-05-31,95.13,96.38,95.13,95.75,37.2819,18000,0.0,1.0
2000-06-01,97.11,97.31,97.11,97.31,37.8893,3500,0.0,1.0
2000-06-02,101.70,102.40,101.70,102.40,39.8712,14700,0.0,1.0
2000-06-05,102.10,103.10,101.90,102.00,39.7154,24200,0.0,1.0
2000-06-06,103.60,104.80,102.70,103.00,40.1048,75900,0.0,1.0
2000-06-07,102.40,103.10,101.40,103.10,40.1437,65500,0.0,1.0
2000-06-08,104.30,104.30,102.90,102.90,40.0659,45700,0.0,1.0


In [0]:
# Calculate exponential weighted moving averages

iwm_signal = create_signal(spy, 10, 40, 0.01)
print('Signal:', iwm_signal.groupby('signal').size())
print('Trend Cross:', iwm_signal.groupby(['cross','signal']).size())
iwm_signal[100:150]

Signal: signal
-1    1779
 0    1206
 1    2421
dtype: int64
Trend Cross: cross  signal
0      -1        1699
        0        1126
        1        2421
1      -1          80
        0          80
dtype: int64


,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient,date,pct_returns,ema_s,ema_l,ema_ratio,signal,cross,buy_price,cross_price,signal_size,cross_size
timestamp,,,,,,,,,,,,,,,,,,,
1998-05-28,109.8750,110.3750,109.0625,110.1250,74.8489,4907600,0.000,1.0,1998-05-28,0.004561,75.225109,75.033622,0.002552,0,0,0.0000,0.000000,0.0,0
1998-05-29,110.6250,110.8125,109.0312,109.0312,74.1055,4772600,0.000,1.0,1998-05-29,-0.009932,75.021543,74.988070,0.000446,0,0,0.0000,0.000000,0.0,0
1998-06-01,108.9687,110.2187,108.5625,109.5312,74.4454,6092200,0.000,1.0,1998-06-01,0.004587,74.916790,74.961444,-0.000596,-1,1,0.0000,74.916790,0.0,12
1998-06-02,110.0000,110.3437,109.1562,109.6250,74.5091,6701100,0.000,1.0,1998-06-02,0.000856,74.842665,74.939257,-0.001289,-1,0,0.0000,0.000000,0.0,0
1998-06-03,109.8750,110.1875,107.8750,107.8750,73.3197,6445100,0.000,1.0,1998-06-03,-0.015963,74.565762,74.859838,-0.003928,-1,0,0.0000,0.000000,0.0,0
1998-06-04,108.2500,110.0625,108.0625,109.8750,74.6790,6566300,0.000,1.0,1998-06-04,0.018539,74.586351,74.850972,-0.003535,-1,0,0.0000,0.000000,0.0,0
1998-06-05,110.3750,112.0000,109.8750,112.0000,76.1233,8463200,0.000,1.0,1998-06-05,0.019340,74.865796,74.913333,-0.000635,-1,0,0.0000,0.000000,0.0,0
1998-06-08,111.9375,112.4687,111.6562,111.8750,76.0384,4159900,0.000,1.0,1998-06-08,-0.001115,75.078997,74.968463,0.001474,0,1,0.0000,75.078997,0.0,12
1998-06-09,111.7187,112.4218,111.4062,112.2187,76.2720,2725300,0.000,1.0,1998-06-09,0.003072,75.295906,75.032324,0.003513,0,0,0.0000,0.000000,0.0,0


In [0]:
iwm_signal.describe().T

,count,mean,std,min,25%,50%,75%,max
open,5406.0,1.507627e+02,5.266238e+01,67.950000,1.140644e+02,1.328596e+02,1.839300e+02,2.960400e+02
high,5406.0,1.516505e+02,5.271521e+01,70.000000,1.148025e+02,1.336800e+02,1.844275e+02,2.963100e+02
low,5406.0,1.497596e+02,5.260083e+01,67.100000,1.132000e+02,1.317628e+02,1.828850e+02,2.937600e+02
close,5406.0,1.507530e+02,5.267196e+01,68.110000,1.140050e+02,1.327750e+02,1.838825e+02,2.958600e+02
adjusted_close,5406.0,1.268769e+02,6.000392e+01,55.006300,8.517663e+01,1.013340e+02,1.650708e+02,2.944263e+02
volume,5406.0,1.034399e+08,9.961119e+07,67900.000000,3.557735e+07,7.537895e+07,1.437105e+08,8.708580e+08
dividend_amount,5406.0,1.105340e-02,9.520128e-02,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,1.435400e+00
split_coefficient,5406.0,1.000000e+00,0.000000e+00,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
pct_returns,5405.0,3.479251e-04,1.210438e-02,-0.096644,-4.845036e-03,6.476799e-04,5.994665e-03,1.396077e-01
ema_s,5406.0,1.266894e+02,5.981117e+01,58.000984,8.521818e+01,1.011044e+02,1.643440e+02,2.909150e+02


In [0]:
iwm_signal.groupby('signal')['pct_returns','ema_ratio'].describe().T

signal                      -1            0            1
pct_returns count  1779.000000  1205.000000  2421.000000
            mean     -0.000142     0.000195     0.000784
            std       0.017188     0.009241     0.008202
            min      -0.096644    -0.057193    -0.041915
            25%      -0.009585    -0.004782    -0.003026
            50%       0.000455     0.000529     0.000730
            75%       0.008766     0.005360     0.005035
            max       0.139608     0.046713     0.058076
ema_ratio   count  1779.000000  1206.000000  2421.000000
            mean     -0.019036     0.005454     0.019934
            std       0.019877     0.002928     0.007709
            min      -0.132593     0.000000     0.010036
            25%      -0.025084     0.002862     0.013823
            50%      -0.013442     0.005768     0.018414
            75%      -0.005485     0.008030     0.024090
            max      -0.000001     0.009996     0.054933

In [0]:
chart_signal(iwm_signal, 'IWM')

Loading BokehJS ...